# Introduction
In this notebook, we will load serialized ContactSequence files from the "examples" directory. See the Readme file of this directory for more information about this files: https://github.com/loco-3d/multicontact-api/tree/master/examples

First we will plot various data stored in the file like CoM trajectory or contact placement, then we will visualize the wholebody motion.

## Displaying a joint trajectory

For this section you will need two additionnal depencies: the simplifed Talos model and the Meshcat 3D visualizer:

* install the binary package `robotpkg-py3\*-example-robot-data` (see http://robotpkg.openrobots.org/robotpkg-wip.html if you haven't configured robotpkg on your system yet
* `pip3 install --user meshcat`

First, let's load the contact sequence from the file. Feel free to change the name of the file loaded to any of the files in the example folder, but be sure to use the `_WB` suffix: the other files do not store joint trajectories.

In [2]:
from multicontact_api import ContactSequence
cs = ContactSequence()
cs.loadFromBinary("../examples/walk_20cm_WB.cs")

print("Number of contact phases: ", cs.size())
print("Duration of the motion: ", cs.contactPhases[-1].timeFinal)

Number of contact phases:  23
Duration of the motion:  35.00000000000026


Then let's load the robot model in Pinocchio and initialize the viewer:

In [6]:
from ipywidgets import interact
from example_robot_data import load
from pinocchio.visualize import MeshcatVisualizer

robot = load('talos')
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.viewer.jupyter_cell()

q0 = robot.q0
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


After the initialization you should get a link to open meshcat in your browser, and you should see Talos standing in it's reference configuration. 

Now we are simply going to retrieve the complete joint trajectory from the file (see the previous section), discretize it and send the configurations to the viewer.

In [8]:
import curves

# concatenate the joint trajectories of each phases in a single curve object
q_t = cs.concatenateQtrajectories()
DT = 0.04 # 25 FPS

t = 0.
t_end = q_t.max()

while t <= t_end:
    viz.display(q_t(t))
    t += DT

